In [1]:
  #!/usr/bin/env python

try:
    import gi
    gi.require_version('NumCosmo', '1.0')
    gi.require_version('NumCosmoMath', '1.0')
except:
    pass

import math
import matplotlib.pyplot as plt
from gi.repository import GObject
from gi.repository import NumCosmo as Nc
from gi.repository import NumCosmoMath as Ncm
import numpy as np
from astropy.io import fits
from astropy.table import Table
import sys
sys.path.insert(0,'../../scripts')

import pyccl as ccl

from nc_ccl import create_nc_obj, ccl_cosmo_set_high_prec

Ncm.cfg_init()
Ncm.cfg_set_log_handler (lambda msg: sys.stdout.write (msg) and sys.stdout.flush ())

from IPython.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

initialize the `Cosmological models`

In [2]:
#CCL cosmology
cosmo_ccl = ccl.Cosmology(Omega_c = 0.30711 - 0.048254, Omega_b = 0.048254, h = 0.677, sigma8 = 0.8822714165197718, n_s=0.96, Omega_k = 0, transfer_function='eisenstein_hu')
#ccl_cosmo_set_high_prec (cosmo_ccl)

cosmo_numcosmo, dist, ps_lin, ps_nln, hmfunc = create_nc_obj (cosmo_ccl)

psf = hmfunc.peek_psf ()

# Define proxy modelling
Use a mass proxy, define the probability for observing a proxy given a mass and redhsift

$$
P(\log\lambda|M,z) = N(\mu(M,z), \sigma^2(M,z))
$$
the mean is
$$
\mu(M,z) = \mu_0 + a_\mu^M\log_{10}\frac{M}{M_0} + a_\mu^z\log_{10}\frac{1+z}{1+z_0} 
$$
variance is
$$
\sigma(M,z) = \sigma_0 + a_\sigma^M\log_{10}\frac{M}{M_0} + a_\sigma ^z\log_{10}\frac{1+z}{1+z_0} 
$$

In [3]:
#CosmoSim_proxy model
#M_0, z_0
theta_pivot = [3e14/0.71, 0.6]
#\mu_0, a_\mu^z, a_\mu^M
theta_mu = [3.19, -0.7, 2]
#\sigma_0, a_\sigma^z, a_\sigma^M
theta_sigma = [0.33, 0.,-0.08]
#Richness object

area = (0.25)*4*np.pi / 100.0
lnRl = 1.0
lnRu = 2.0
zl = 0.25
zu = 1.0

#Numcosmo_proxy model
cluster_z = Nc.ClusterRedshift.new_from_name("NcClusterRedshiftNodist{'z-min': <%20.15e>, 'z-max':<%20.15e>}" % (zl, zu))
'''
cluster_m = Nc.ClusterMass.new_from_name("NcClusterMassAscaso{'M0':<%20.15e>,'z0':<%20.15e>,'lnRichness-min':<%20.15e>, 'lnRichness-max':<%20.15e>}" % (3e14/(0.71),0.6, lnRl, lnRu))
cluster_m.param_set_by_name('mup0', 3.19)
cluster_m.param_set_by_name('mup1', 2/np.log(10))
cluster_m.param_set_by_name('mup2', -0.7/np.log(10))
cluster_m.param_set_by_name('sigmap0', 0.33)
cluster_m.param_set_by_name('sigmap1', -0.08/np.log(10))
cluster_m.param_set_by_name('sigmap2', 0/np.log(10))
'''

cluster_m = Nc.ClusterMass.new_from_name("NcClusterMassNodist{'lnM-min':<%20.15e>, 'lnM-max':<%20.15e>}" % (math.log(10)*np.log10(1e14),math.log(10)*np.log10(1e16)))

initialize the `ClusterAbundance` object

In [4]:
#Numcosmo Cluster Abundance

#First we need to define the multiplicity function here we will use the tinker
mulf = Nc.MultiplicityFuncTinker.new()
mulf.set_linear_interp (True)
mulf.set_mdef(Nc.MultiplicityFuncMassDef.CRITICAL)
mulf.set_Delta(200)
#Second we need to construct a filtered power spectrum 

hmf = Nc.HaloMassFunction.new(dist,psf,mulf)
hmf.set_area(area)

ca = Nc.ClusterAbundance.new(hmf,None)
mset = Ncm.MSet.new_array([cosmo_numcosmo,cluster_m,cluster_z])

In [5]:
ncount = Nc.DataClusterNCount.new (ca, "NcClusterRedshiftNodist", "NcClusterMassNodist")
ncount.catalog_load ("ncount_nodist.fits")

In [6]:
cosmo_numcosmo.props.Omegac_fit = True
cosmo_numcosmo.props.w0_fit = True
#cluster_m.props.mup0_fit = True
mset.prepare_fparam_map ()

In [7]:
ncount.set_binned (True)

In [8]:
dset = Ncm.Dataset.new ()
dset.append_data (ncount)

In [9]:
lh = Ncm.Likelihood (dataset = dset)
fit = Ncm.Fit.new (Ncm.FitType.NLOPT, "ln-neldermead", lh, mset, Ncm.FitGradType.NUMDIFF_FORWARD)

In [10]:
Ncm.func_eval_set_max_threads (2)
Ncm.func_eval_log_pool_stats ()
print (Ncm.func_eval_log_pool_stats ())


init_sampler = Ncm.MSetTransKernGauss.new (0)
init_sampler.set_mset (mset)
init_sampler.set_prior_from_mset ()
init_sampler.set_cov_from_rescale (1.0)

nwalkers = int (math.ceil (500)) #500
ssize    = 150000 #1000000

walker = Ncm.FitESMCMCWalkerAPES.new (nwalkers, mset.fparams_len ())

esmcmc = Ncm.FitESMCMC.new (fit, nwalkers, init_sampler, walker, Ncm.FitRunMsgs.SIMPLE)
esmcmc.set_nthreads (2)

esmcmc.set_data_file ("ncount_nodist_binned.fits")

esmcmc.start_run ()
esmcmc.run(ssize / nwalkers)
esmcmc.end_run ()

esmcmc.mean_covar ()
fit.log_covar ()


# NcmThreadPool:Unused:      0
# NcmThreadPool:Max Unused:  2
# NcmThreadPool:Running:     0
# NcmThreadPool:Unprocessed: 0
# NcmThreadPool:Unused:      2
# NcmThreadPool:Unused:      0
# NcmThreadPool:Max Unused:  2
# NcmThreadPool:Running:     0
# NcmThreadPool:Unprocessed: 0
# NcmThreadPool:Unused:      2
None
#----------------------------------------------------------------------------------
# NcmFitESMCMC: Starting Ensamble Sampler Markov Chain Monte Carlo.
#   Number of walkers: 0500.
#   Number of threads: 0002.
#   Using MPI:         no - use MPI enabled but no slaves available.
#----------------------------------------------------------------------------------
# NcmFitESMCMC: Skipping 15000 points (30.000000 iterations), will start at 15001-th point.
#----------------------------------------------------------------------------------
# NcmFitESMCMC: validating catalog rows [14500, 15000)


: 

: 

In [ ]:
ntests   = 100.0
nwalkers = 300
burnin   = 10
mcat = Ncm.MSetCatalog.new_from_file_ro ("ncount_nodist_binned.fits", nwalkers * burnin)

mcat.log_current_chain_stats ()
mcat.calc_max_ess_time (ntests, Ncm.FitRunMsgs.FULL);
mcat.calc_heidel_diag (ntests, 0.0, Ncm.FitRunMsgs.FULL);

mset.pretty_log ()
mcat.log_full_covar ()
mcat.log_current_stats ()

be, post_lnnorm_sd = mcat.get_post_lnnorm ()
lnevol, glnvol = mcat.get_post_lnvol (0.6827)

Ncm.cfg_msg_sepa ()
print ("# Bayesian evidence:                                 % 22.15g +/- % 22.15g" % (be, post_lnnorm_sd))
print ("# 1 sigma posterior volume:                          % 22.15g" % lnevol)
print ("# 1 sigma posterior volume (Gaussian approximation): % 22.15g" % glnvol)

# NcmMSetCatalog: Current skfac:  1.0046       1.0301       1.0246     
# NcmMSetCatalog: Maximal Shrink factor = 1.03024               
#----------------------------------------------------------------------------------
# NcmMSetCatalog: Computing the constant break point for parameter `0', sample size `24':
# NcmMSetCatalog: Constant break point at `5':
#----------------------------------------------------------------------------------
# NcmMSetCatalog: Calculating catalog effective sample size from chain 24 => 0 using 100 blocks:
# NcmMSetCatalog: - best cutoff time:         0   
# NcmMSetCatalog: - total number of points:   24   (12000)
# NcmMSetCatalog: - number of points left:    24   (12000)
# NcmMSetCatalog: - worst parameter:          NcHICosmo:w0[02]
# NcmMSetCatalog: - worst parameter ess:      7.15
# NcmMSetCatalog: - worst parameter ar order: 1   
# NcmMSetCatalog: - ess's:                    24.00 10.97 7.15
#---------------------------------------------------------------